In [1]:
import sys
sys.path.append("..")
import argparse
import numpy as np
import dgl
from dgl import DGLGraph
import torch
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from collections import Counter
import pickle
import h5py
import random
import glob2
import seaborn as sns

import train
import models

%load_ext autoreload
%autoreload 2

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
device = train.get_device()

Using backend: pytorch
/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Detected an old version of PyTorch. Suggest using torch>=1.5.0 for the best experience.
  return warnings.warn(message, category=category, stacklevel=1)
/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [3]:

model_name = "GraphConv"
normalize_weights = "log_per_cell"
node_features = "scale"
same_edge_values = False
edge_norm = True
hidden_relu = False
hidden_bn = False
n_layers = 1
pca_size = 50
epochs = 10
batch_size = 128
hidden_dim = 200
hidden = [300]
nb_genes = 3000
activation = F.relu
for category in ["real_data","balanced_data", "imbalanced_data", 
                ]:
    results = pd.DataFrame()
    path= ".."
    if category in ["balanced_data", "imbalanced_data"]:
        files = glob2.glob(f'{path}/R/simulated_data/{category}/*.h5')
        files = [f[len(f"{path}/R/simulated_data/{category}/"):-3] for f in files]

    else:
        files = glob2.glob(f'{path}/real_data/*.h5')
        files = [f[len(f"{path}/real_data/"):-3] for f in files]

    print(files)
    
    for dataset in files:
        if category in ["balanced_data", "imbalanced_data"]:
            data_mat = h5py.File(f"{path}/R/simulated_data/{category}/{dataset}.h5","r")
        else:
            data_mat = h5py.File(f"{path}/real_data/{dataset}.h5","r")
        print(f">> {dataset}")

        Y = np.array(data_mat['Y'])
        X = np.array(data_mat['X'])

        genes_idx, cells_idx = train.filter_data(X, highly_genes=nb_genes)
        X = X[cells_idx][:, genes_idx]
        Y = Y[cells_idx]

        n_clusters = len(np.unique(Y))
        for pca_size in [5, 10, 50, 100, 300, 500]:
            graph = train.make_graph(
                X,
                Y,
                dense_dim=pca_size,
                node_features=node_features,
                normalize_weights=normalize_weights,
                edge_norm =edge_norm
            )

            labels = graph.ndata["label"]
            train_ids = np.where(labels != -1)[0]

            sampler = dgl.dataloading.MultiLayerFullNeighborSampler(n_layers)

            dataloader = dgl.dataloading.NodeDataLoader(
                graph,
                train_ids,
                sampler,
                batch_size=batch_size,
                shuffle=True,
                drop_last=False,
                num_workers=1,
            )
            print(
                f"INPUT: {model_name}  {hidden_dim}, {hidden}, {same_edge_values}, {edge_norm}"
            )
            t1 = time.time()

            for run in range(3):
                t_start = time.time()
                torch.manual_seed(run)
                torch.cuda.manual_seed_all(run)
                np.random.seed(run)
                random.seed(run)

                model = models.GCNAE(
                    in_feats=pca_size,
                    n_hidden=hidden_dim,
                    n_layers=n_layers,
                    activation=activation,
                    dropout=0.1,
                    hidden=hidden,
                    hidden_relu=hidden_relu,
                    hidden_bn=hidden_bn,
                ).to(device)
                if run == 0:
                    print(f">", model)

                optim = torch.optim.Adam(model.parameters(), lr=1e-5)

                scores = train.train(model, optim, epochs, dataloader, n_clusters, plot=False,
                                    cluster=["KMeans", "Leiden"])
                scores["dataset"] = dataset
                scores["run"] = run
                scores["nb_genes"] = nb_genes
                scores["pca_size"] = pca_size

                results = results.append(scores, ignore_index = True)

                results.to_pickle(
                    f"../output/pickle_results/{category}/{category}_pca_size.pkl")
                print("Done")

['Muraro', 'Quake_Smart-seq2_Diaphragm', 'worm_neuron_cell', 'Quake_10x_Bladder', 'mouse_bladder_cell', 'Young', '10X_PBMC', 'Quake_10x_Limb_Muscle', 'Adam', 'Quake_Smart-seq2_Trachea', 'Quake_Smart-seq2_Limb_Muscle', 'Quake_10x_Spleen', 'Quake_Smart-seq2_Lung', 'mouse_ES_cell', 'Romanov']
>> Muraro


/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


INPUT: GraphConv  200, [300], False, True
> GCNAE(
  (dropout): Dropout(p=0.1, inplace=False)
  (layer1): WeightedGraphConv(in=5, out=200, normalization=both, activation=<function relu at 0x7f60433410e0>)
  (decoder): InnerProductDecoder()
  (encoder): Sequential(
    (0): Linear(in_features=200, out_features=300, bias=True)
  )
)


/opt/conda/lib/python3.7/site-packages/umap/__init__.py:9: UserWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn("Tensorflow not installed; ParametricUMAP will be unavailable")


ARI 0.8688, 0.5528296828269958
Done


...

/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


INPUT: GraphConv  200, [300], False, True
> GCNAE(
  (dropout): Dropout(p=0.1, inplace=False)
  (layer1): WeightedGraphConv(in=500, out=200, normalization=both, activation=<function relu at 0x7f60433410e0>)
  (decoder): InnerProductDecoder()
  (encoder): Sequential(
    (0): Linear(in_features=200, out_features=300, bias=True)
  )
)



ARI 0.0035, 0.05063889920711517
Done



ARI 0.007, 0.04489849880337715
Done



ARI 0.0135, 0.03165089339017868
Done


In [ ]:
results = pd.read_pickle(
                            f"output/pickle_results/{category}/{category}_pca_size.pkl"
                        )

In [ ]:
results.mean()#0.7658

In [4]:
results.groupby("dataset").mean()

,ae_end,kmeans_ari,kmeans_cal,kmeans_nmi,kmeans_sil,kmeans_time,leiden_ari,leiden_cal,leiden_nmi,leiden_sil,leiden_time,nb_genes,pca_size,run
dataset,,,,,,,,,,,,,,
data_-1c16,1.615143e+09,0.012700,1267.213737,0.041878,0.066065,7.828056,0.002428,2128.792430,0.012489,0.121434,13.961976,3000.0,160.833333,1.0
data_-1c4,1.615149e+09,0.003722,2202.030311,0.006422,0.176113,7.051438,0.216733,1373.627802,0.299128,0.125094,14.061370,3000.0,160.833333,1.0
data_-1c8,1.615147e+09,0.010333,1525.917156,0.022078,0.096756,7.567885,0.124417,1233.831733,0.190856,0.110039,14.941466,3000.0,160.833333,1.0
data_0c16,1.615136e+09,0.008444,893.999101,0.031872,0.058457,8.082729,0.001483,1587.888701,0.009056,0.099116,14.909415,3000.0,160.833333,1.0
data_0c4,1.615138e+09,0.003883,1667.757436,0.005806,0.152249,6.968370,0.157461,1079.013216,0.213806,0.100908,14.600068,3000.0,160.833333,1.0
data_0c8,1.615133e+09,0.011528,1114.501905,0.022128,0.083082,7.491724,0.081544,1019.176250,0.119194,0.092757,14.230310,3000.0,160.833333,1.0
data_1.5c16,1.615137e+09,0.003761,365.541317,0.023217,0.049440,8.317576,0.001200,670.912927,0.008633,0.062348,15.042789,3000.0,160.833333,1.0
data_1.5c4,1.615140e+09,0.009383,843.450975,0.009806,0.104573,7.192750,0.087861,570.430881,0.111333,0.067274,15.238115,3000.0,160.833333,1.0
data_1.5c8,1.615141e+09,0.012828,550.599925,0.022711,0.061328,7.986625,0.013700,584.267047,0.022822,0.055548,16.306325,3000.0,160.833333,1.0
